# Use FB Prophet for Multivariate Time-series Forecasting: Six Group merchant for sum of all transactions

Prophet is an open source library developed by Facebook which aims to make time-series forecasting easy and scalable. It is a type of a generalized additive model (GAM), which uses regression model with potentially non-linear smoothers. It is called additive because it addes multiple decomposed parts to explain some trends. For example, Prophet uses the following components: 

$$ y(t) = g(t) + s(t) + h(t) + e(t) $$

where,  
$g(t)$: Growth. Big trend. Non-periodic changes.   
$s(t)$: Sesonality. Periodic changes (e.g. weekly, yearly, etc.) represented by Fourier Series.  
$h(t)$: Holiday effect that represents irregular schedules.   
$e(t)$: Error. Any idiosyncratic changes not explained by the model. 

# Table of Contents 
1. [Prepare Data](#prep)
2. [Get Data](#data)
3. [Data Processing](#processing)
4. [Train Test Split](#split)
5. [Baseline Model](#baseline)
6. [Multivariate Model](#multivariate)
7. [References](#References)

# Step 1: Install and Import Libraries

<a id=prep></a>
# 1. Prepare Data

The goal of the time series model is to predict the six group merchant transactions. Prophet requires at least two columns as inputs: a ds column and a y column.

 * The ds column has the time information. Currently we have the date as the index, so we name this index as ds.
 * The y column has the time series transaction values. In this example, because we are predicting six group merchant transactions, the column name for the transactions is named y.

In [ ]:
# Prophet model for time series forecast
from prophet import Prophet

# Data processing
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
# import the math module
import math
import sklearn.metrics

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import miceforest as mf
# Hyperparameter tuning
import itertools
import json
from prophet.diagnostics import cross_validation, performance_metrics

# Step 2: Get Data

In [ ]:
# you need to change the file path
#data_path = "../../data/raw/Time_Series_Merchants_Transactions_Anonymized.csv"
#data_path = "data/Time_Series_Merchants_Transactions_Anonymized.csv"
data_path = "../data/ninety.csv"
#data_path = "data/ninetyfive.csv"
df_merchant_transactions = pd.read_csv(data_path)
merchant_names = df_merchant_transactions.iloc[:,0].values
df_merchant_transactions = df_merchant_transactions.drop(columns='Merchant Name')
my_input = ['ds', 'y'] 
my_input.extend(merchant_names) 
print(my_input)

In [ ]:
# replacing columns names with standard date format
stddates = pd.date_range(start='2020-08', end='2022-10', freq="M")
df_merchant_transactions = pd.DataFrame(df_merchant_transactions.values)
df_merchant_transactions = df_merchant_transactions.T
#stddates

In [ ]:
data_path_sum = "../data/data_clean_sum.csv"
df_merchant_transactions_sum = pd.read_csv(data_path_sum)
df_merchant_transactions_sum

In [ ]:
ds = [df_merchant_transactions_sum,df_merchant_transactions]
df_merchant_transactions_concat = pd.concat(ds, join='outer', axis=1)
df_merchant_transactions_concat

# Step 3: Data Processing

Step 3 transforms the dataset into a time series modeling dataset. 


Prophet requires at least two columns as inputs: a `ds` column and a `y` column.
* The `ds` column has the time information. Currently we have the date as the index, so we reset the index and rename `date` to `ds`.
* The y column has the time series values. In this example, because we are predicting the Google closing price, the column name `Merchant 1` is changed to `y`.
* There is no pre-defined name for the additional predictor in prophet, so we can keep the names as is.

In [ ]:
data = pd.DataFrame(df_merchant_transactions_concat.values)
# Change variable names
data.columns = my_input
data.head()

In [ ]:
# Check correlation
data_test = data.drop(columns='ds').apply(pd.to_numeric)
correlation = data_test.corrwith(data_test['y'])#,numeric_only=True)
correlation

In [ ]:
# get heatmap of the correlation
fig, ax = plt.subplots(figsize=(40, 30))
sns.heatmap(data_test.corr(), ax=ax, annot=True)

# Step 4: Train Test Split

In step 4, we will do the train test split. For time series data, usually a threshold date is chosen, then we set the dates before the threshold to be the training dataset and the dates after the threshold to be the testing dataset.

Based on the threshold date (`train_end_date`) we set before, there are data points in the training dataset and data points in the testing dataset.

In [ ]:
# Train test split the date need to be changed 
train_end_date = '2022-04-30'
train = data[data['ds'] <= train_end_date]
test = data[data['ds'] > train_end_date]

# Check the shape of the dataset
print(train.shape)
print(test.shape)

Checking the minimum and maximum values for the train and test dataset separately gave us the starting and ending dates.

In [ ]:
# Check the start and end time of the training and testing dataset
print('The start time of the training dataset is ', train['ds'].min())
print('The end time of the training dataset is ', train['ds'].max())
print('The start time of the testing dataset is ', test['ds'].min())
print('The end time of the testing dataset is ', test['ds'].max())

# Step 5: Baseline Model

In step 5, we will build a univariate baseline model using the default prophet hyperparameters, and fit the model using the training dataset.

## Step 5.1: Build Baseline Model

In [ ]:
# Use the default hyperparameters to initiate the Prophet model
model_baseline = Prophet(interval_width=0.99, seasonality_mode='multiplicative')

# Fit the model on the training dataset
model_baseline.fit(train)

Prophet automatically fits daily, weekly, and yearly seasonalities if the time series is more than two cycles long.

The model information shows that the yearly seasonality and the daily seasonality are disabled. 
* The daily seasonality is disabled because we do not have sub-daily time series. 
* The yearly seasonality is disabled although we have two years of data. 

We will continue with the default values for the baseline model and force the yearly seasonality in the next model to see the impact of the yearly seasonality.

## Step 5.2: Baseline Model Forecast

After making the prediction on the future dataframe, we can plot the result using `.plot`.
* The black dots are the actual values.
* The blue line is the prediction.
* The blue shades are the uncertainty interval. The default value for the uncertainty interval is 80%, so we are using 80% here. The uncertainty interval is calculated based on the assumption that the average frequency and magnitude of trend changes in the future will be the same as the historical data. The historical data trend changes are projected forward to get the uncertainty intervals [1].

In [ ]:
# Create the time range for the forecast
future_baseline = model_baseline.make_future_dataframe(test.shape[0], freq='M')

# Make prediction
forecast_baseline = model_baseline.predict(future_baseline)

# Visualize the forecast
fig = model_baseline.plot(forecast_baseline); # Add semi-colon to remove the duplicated chart
plt.legend(['Actual', 'Prediction', 'Uncertainty interval'])
plt.show()

In addition to the forecast plot, prophet also provides the components plot. 

From the component plot chart, we can see that the no of transactions has an overall upward trend. The weekly seasonality shows that the price tends to be lower at the beginning of the week and higher at the end of the week.

In [ ]:
# Visualize the forecast components
model_baseline.plot_components(forecast_baseline);

## Step 6: Multivariate Model Performance

In [ ]:
def rmse(data, forecast):
    # rmse = sqrt(sklearn.metrics.mean_squared_error(test[merchant_name].values, fitted.values))
    return math.sqrt(sklearn.metrics.mean_squared_error(data, forecast))

rmse(data['y'].iloc[train.shape[0]: train.shape[0] + test.shape[0]], forecast_baseline['yhat'][train.shape[0]: train.shape[0] + test.shape[0]])

In [ ]:
# you need to change the file path
# here the input file is the forecast result using the best model 
# ETS for the 5 months which are a input to the prophet
# multivariate approach and added as regressors for predictions
predict_multivariate = pd.read_csv('Pred_ETS_5months.csv')
del predict_multivariate[predict_multivariate.columns[0]]
# Using DataFrame.insert() to add a column
predict_multivariate.insert(0, "ds", stddates[stddates > train_end_date], True)
predict_multivariate.head()

In [ ]:
dfs1 = [train, predict_multivariate]
# Append the date(ds) column
train1 = pd.concat(dfs1).reset_index(drop=True)
train1.tail(6)

In [ ]:
# Set up parameter grid
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.05, 0.08, 0.5],
    'seasonality_prior_scale': [0.01, 1, 5, 10, 12],
    'seasonality_mode': ['additive', 'multiplicative']
}# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]# Create a list to store MAPE values for each combination
mapes = [] 
# horizon = test period of each fold
horizon = '91 days'
# initial: training period. (optional. default is 3x of horizon)
initial = str(91 * 2) + ' days'  
# period: spacing between cutoff dates (optional. default is 0.5x of horizon)
period = str(91 * 2) + ' days' 

# Use cross validation to evaluate all parameters
for params in all_params:
    # Fit a model using one parameter combination
    m = Prophet(**params).fit(train)  
    # Cross-validation
    df_cv = cross_validation(m, initial=initial, period=period, horizon=horizon)
    # Model performance
    df_p = performance_metrics(df_cv, rolling_window=1)
    # Save model performance metrics
    mapes.append(df_p['mape'].values[0])            
    
# Tuning results
tuning_results = pd.DataFrame(all_params)
tuning_results['mape'] = mapes# Find the best parameters
best_params = all_params[np.argmin(mapes)]
#auto_model = Prophet(changepoint_prior_scale=best_params['changepoint_prior_scale'], 
#                     seasonality_prior_scale=best_params['seasonality_prior_scale'], 
#                     seasonality_mode=best_params['seasonality_mode'])

In [ ]:
print(best_params)# Fit the model using the best parameters

In [ ]:
rmse_merchant_ids=[]
rmse_error = 0

for m in range(10):
    # changes the no of merchants added to regressors starting 
    for l in range(6):
        model_multivariate = Prophet(interval_width=0.95, changepoint_prior_scale=0.05, seasonality_prior_scale=0.01, seasonality_mode='additive') #seasonality_mode='multiplicative')
        merchant_ids=[]
        k = 0
        my_input_shuffle = shuffle(merchant_names)
        correlation = data_test[my_input_shuffle].corrwith(data_test['y'], method='spearman')   #method{‘pearson’, ‘kendall’, ‘spearman’} or callable
        # Add regressor
        for i in range(len(merchant_names)):
            if(k>l): break
            if(correlation[i] > 0.80):
                model_multivariate.add_regressor(my_input_shuffle[i], standardize=False)
                merchant_ids.append(my_input_shuffle[i])
                k+=1

        # Fit the model on the training dataset
        model_multivariate.fit(train)
        df2 = train1[merchant_ids]
        future_multivariate = model_multivariate.make_future_dataframe(6, freq='M')
        dfs = [future_multivariate, df2]
        # Append the regressor values
        future_multivariate = pd.concat(dfs, join='inner', axis=1)
        # Make prediction
        forecast_multivariate = model_multivariate.predict(future_multivariate)
        # Visualize the forecast
        #model_multivariate.plot(forecast_multivariate)
        error_rmse = rmse(data['y'].iloc[21:26], forecast_multivariate['yhat'].iloc[21:26])
        if m == 0: rmse_error = error_rmse
        if rmse_error > error_rmse : 
            rmse_error = error_rmse
            rmse_merchant_ids.append(merchant_ids)
        #print(l,' ',error_rmse)
        #print(merchant_ids)
        del model_multivariate
        del forecast_multivariate
        del future_multivariate
        del df2
        del dfs
print('smallest RMSE error: ', rmse_error)
print('merchant ids: ', rmse_merchant_ids)
#        f.write(' the rmse error is ')
#        f.write(str(error_rmse))
#        f.write('\n') smallest error:  535403.1731782097

In [ ]:
rmse_merchant_ids=[]
rmse_error = 0

model_multivariate = Prophet(interval_width=0.99, changepoint_prior_scale=0.05, seasonality_prior_scale=0.01, seasonality_mode='additive') #seasonality_mode='multiplicative')
merchant_ids=[]
#these are ideal merchant IDS obtained from the above rune which are needed for the multivariate run to get the lowest RMSE error
merchant_ids = ['Merchant 828', 'Merchant 156', 'Merchant 483', 'Merchant 13', 'Merchant 242']
model_multivariate.add_regressor('Merchant 828', standardize=False)
model_multivariate.add_regressor('Merchant 156', standardize=False)
model_multivariate.add_regressor('Merchant 483', standardize=False)
model_multivariate.add_regressor('Merchant 13', standardize=False)
model_multivariate.add_regressor('Merchant 242', standardize=False)

# Fit the model on the training dataset
model_multivariate.fit(train)
df2 = train1[merchant_ids]
future_multivariate = model_multivariate.make_future_dataframe(5, freq='M')
dfs = [future_multivariate, df2]
# Append the regressor values
future_multivariate = pd.concat(dfs, join='inner', axis=1)
# Make prediction
forecast_multivariate = model_multivariate.predict(future_multivariate)
# Visualize the forecast
model_multivariate.plot(forecast_multivariate)
error_rmse = rmse(data['y'].iloc[21:26], forecast_multivariate['yhat'].iloc[21:26])
print('smallest RMSE error: ', error_rmse)
print('merchant ids: ', merchant_ids)

In [ ]:
forecast_multivariate['ds'].iloc[train.shape[0]: train.shape[0] + test.shape[0]], forecast_multivariate['yhat'].iloc[train.shape[0]: train.shape[0] + test.shape[0]]

# References

[1] [Prophet Documentation](https://facebook.github.io/prophet/docs/seasonality,_holiday_effects,_and_regressors.html)